# Analysis Pipeline for Intracranial Data


## STEPS:
##### 1. run the IMPORT AND LOAD DATA section.

##### 2. fill in the values extracted from the plot from step 1 for following variables:


Let's go

In [15]:
trigger_ch.info

<Info | 7 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F7, F3, Fz, F4, F8, T7, C3, Cz, C4, T8, P7, P3, Pz, ...
 chs: 26 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2022-08-03 15:52:44 UTC
 nchan: 26
 projs: []
 sfreq: 256.0 Hz
>

In [1]:
import os
import numpy as np
import mne
from mne.utils import _TempDir
import matplotlib.pyplot as plt
import pyqtgraph as pg
import pyqtgraph as plotWidget
from get_trigger_indexes_photodiode import normalized
%matplotlib qt
from pd_parser.parse_pd import _read_raw, _to_tsv
import time
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
from datetime import datetime
%matplotlib qt


# DEFINE PATHS
data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE'
# PT_id = 
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')
data_raw_file = os.path.join(data_path, 'EEG_1051662_test_AudCatLoc_latency_w_microphone.edf')
# DEFINE PATHS ENDS

# DEFINE NAMES
Exper = 'CatLoc'
PT_id = 'PAT_3066'
print(PT_id)
fig = plt.figure(figsize=(100, 2))
# DEFINE NAMES ENDS

# LOAD TRIGGER and MIC DATA 
# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf') #PAT_3066_EEG_695997_Anon_FLM.edf
# trigger_ch = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
trigger_ch = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-'])
print('*   Trigger channel length: ',trigger_ch.__len__)
print(trigger_ch.info.ch_names)
# picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='MKR+')
picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='F7') 
trigger_ch2 = trigger_ch.get_data(picks=picks)
picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp='F8') 
mic_channel = trigger_ch.get_data(picks=picks)
trigger_ch2 = normalized(trigger_ch2[0].T)
mic_channel = normalized(mic_channel[0].T)
# LOAD TRIGGER and MIC DATA ENDS


# VISUALIZE
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))
# fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
pg.setConfigOption('background','w')
PlotWidget = pg.plot(time_s,trigger_ch2,title='trigger',pen='r')
PlotWidget.plot(time_s,mic_channel,title='mic',pen='blue')
# VISUALIZE ENDS


PAT_3066


objc[41252]: Class QCocoaPageLayoutDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x1246d85c0) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/lib/QtPrintSupport.framework/Versions/5/QtPrintSupport (0x12d793478). One of the two will be used. Which one is undefined.
objc[41252]: Class QCocoaPrintPanelDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x1246d8638) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/lib/QtPrintSupport.framework/Versions/5/QtPrintSupport (0x12d7934f0). One of the two will be used. Which one is undefined.
objc[41252]: Class QCocoaApplicationDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x1246d8340) and /Library/Frameworks/Python.framework

Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/EEG_1051662_test_AudCatLoc_latency_w_microphone.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*   Trigger channel length:  <bound method BaseRaw.__len__ of <RawEDF | EEG_1051662_test_AudCatLoc_latency_w_microphone.edf, 26 x 36608 (143.0 s), ~30 kB, data not loaded>>
['Fp1', 'Fp2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T7', 'C3', 'Cz', 'C4', 'T8', 'P7', 'P3', 'Pz', 'P4', 'P8', 'O1', 'O2', 'F9', 'FT9', 'P9', 'F10', 'FT10', 'P10', 'MKR+']
(36608,)


### Extract the PD Trigger

In [ ]:
# GET PHOTODIODE TRIGGERS
Exper = 'MIC and Trig Test'
print('\n\n*****----------------  ',Exper,'  ----------------*****\n' )

t_onset_s= 6000
t_offset_s= 35000
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75, trigger_channel='F7')



### SAVE DATA:

In [ ]:

date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

#     # 28840


### VALIDATION
Assert that the trigger onset and offset are a pair and the onset is before the offset

In [3]:
# VALIDATION
# index_on_true=index_onset
# index_off_true=index_offset
# index_onset = list(index_onset)
# index_offset = list(index_offset)
# validity = [] # 0 for invalid, 1 for valid
# for i in range(min([len(index_on_true),len(index_off_true)])):
#     tempe=True
#     # print(i)
#     # if i >= min([len(index_on_true),len(index_off_true)]):
#     #     validity.append(0)
#     if index_onset[i] < index_offset[i]:
#         while tempe:
#             while index_onset[i] < index_offset[i]:
#                 if index_onset[i+1] <index_offset[i]:
#                     # print(index_onset[i])
#                     index_onset.pop(i)
#                     validity.append(0)
#                     tempe=False
#                     break
#                 else:
#                     validity.append(1)
#                     tempe=False
#                     break
#     else:
#         while tempe:
#             while index_onset[i] > index_offset[i]:
#                 if index_onset[i+1] >index_offset[i]:
#                     index_offset.pop(i)
#                     validity.append(0)
#                     tempte=False
#                 else:
#                     validity.append(1)
#                     tempe=False
#                     break



# if len(index_offset)!=len(index_onset):
#     if len(index_offset)>len(index_onset):
#         index_onset.append(0)
# diff_oo=[index_offset[i]-index_onset[i] for i in range(len(index_offset))]
# assert np.min(diff_oo)>=0
# # print(validity, len(validity),len(index_onset),len(index_offset))
# validity=np.array(validity)

In [4]:

# VISUALIZE
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
# pg.plot(time_s,trigger_ch2)
# pg.plot(time_s,index_onset)
# plt.close(fig)
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))
plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
plotWidget.plot(time_s,trigger_ch2)
plotWidget.plot(index_offset,trigger_ch2[index_offset],pen=None, symbol='o')
plotWidget.plot(index_onset,trigger_ch2[index_onset],pen=None, symbol='x')
# plotWidget.plot(index_onset[validity==False],trigger_ch2[index_onset[validity==False]],pen=None, symbol='x')

# plotWidget.show()

(2181120,)


### Save The Data

In [6]:
from datetime import datetime
date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()


### FINISHED HERE: 
##### Everything below is draft

In [13]:

## CHECKS ALL CHANNELS
# for ch_nam in trigger_ch.info.ch_names:
#     picks = mne.pick_channels_regexp(trigger_ch.info.ch_names, regexp=ch_nam) 
#     trigger_ch2 = trigger_ch.get_data(picks=picks)
#     trigger_ch2 = trigger_ch2[0].T
#     time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))

#     # VISUALIZE
#     # fig = plt.figure(figsize=(100, 2))
#     print(np.shape(trigger_ch2))
#     import pyqtgraph as pg
#     pg.plot(time_s,trigger_ch2,title=ch_nam)
#     pg.parametertree
## CHECKS ALL CHANNELS ENDS


#     # 30446
# # index_on_true=index_onset
# # index_off_true=index_offset

# Exper = 'FLM_audio'
# print('\n\n*****----------------',Exper,'  ----------------*****\n' )
# t_onset_s= 1325000
# t_offset_s= 2130000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.72)

# date_txt=str(datetime.date(datetime.now()))
# name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
# with open(os.path.join(data_path,name_txt ), "w") as textfile:
#     textfile.write('onsets;offsets\n')
#     XY = [i for i in zip(index_onset, index_offset)]
#     for x, y in XY:
#         textfile.write(str(x)+';'+str(y))
#         textfile.write('\n')
#     textfile.close()

    # 28840
    # 30446
# index_on_true=index_onset
# index_off_true=index_offset



# Exper = 'FLM_reading'
# print('\n\n*****----------------',Exper,'  ----------------*****\n' )
# t_onset_s= 2702000
# t_offset_s= 3315000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75)

# date_txt=str(datetime.date(datetime.now()))
# name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
# with open(os.path.join(data_path,name_txt ), "w") as textfile:
#     textfile.write('onsets;offsets\n')
#     XY = [i for i in zip(index_onset, index_offset)]
#     for x, y in XY:
#         textfile.write(str(x)+';'+str(y))
#         textfile.write('\n')
#     textfile.close()

    # 28840
    # 30446
# index_on_true=index_onset
# index_off_true=index_offset
# t_onset_s= 1022000
# t_offset_s= 1900000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75)

# index_on_true=index_onset
# index_off_true=index_offset

# t_onset_s= 1022000
# t_offset_s= 1900000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75)

# index_on_true=index_onset
# index_off_true=index_offset